In [318]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import model_selection
from sklearn.metrics import classification_report 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from pandas.plotting import scatter_matrix #Para monstrar un matriz de scater con todos los valores
import matplotlib.pyplot as plt
import seaborn as sb

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [319]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [320]:
dataframe = pd.read_csv(r"./SDC2/kidney_disease.csv") 

In [321]:
dataframe.head()

,id,age,bp,sg,al,su,rbc,pc,pcc,ba,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,...,38,6000,NaN,no,no,no,good,no,no,ckd
2,2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,...,31,7500,NaN,no,yes,no,poor,no,yes,ckd
3,3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,...,35,7300,4.6,no,no,no,good,no,no,ckd


In [322]:
dataframe = dataframe.drop(['id'],1)
dataframe.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,...,38,6000,NaN,no,no,no,good,no,no,ckd
2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,...,31,7500,NaN,no,yes,no,poor,no,yes,ckd
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,...,35,7300,4.6,no,no,no,good,no,no,ckd


In [323]:
#dataframe.bgr.str.replace('\t?','')
#dataframe.bu.str.replace('\t?','')
#dataframe.sc.str.replace('\t?','')
#dataframe.sod.str.replace('\t?','')
#dataframe.pot.str.replace('\t?','')
#dataframe.hemo.str.replace('\t?','')

#dataframe['bgr'] = dataframe.bgr.str.replace('?','nan')
#dataframe['bu'] = dataframe.bu.str.replace('?','nan')
#dataframe['sc'] = dataframe.sc.str.replace('?','nan')
#dataframe['sod'] = dataframe.sod.str.replace('?','nan')
#dataframe['pot'] = dataframe.pot.str.replace('?','nan')
#dataframe['hemo'] = dataframe.hemo.str.replace('?','nan')
dataframe['pcv'] = dataframe.pcv.str.replace('?','nan')
dataframe['wc'] = dataframe.wc.str.replace('?','nan')
dataframe['rc'] = dataframe.rc.str.replace('?','nan')
#for index, row in dataframe.iterrows():
#    print(index, row.pcv)

#dataframe.pcv.str.replace('	?','nan')
#dataframe.wc.str.replace('\t?','0')
#dataframe.rc.str.replace('\t?','0')

In [324]:
dataframe.shape

(400, 25)

In [325]:
dataframe.dropna(axis=0, subset=['rbc','pc','pcc','ba','htn','dm','cad','appet','pe','ane'], inplace = True)

In [326]:
dataframe.shape

(233, 25)

In [327]:
dataframe.loc[dataframe['classification'] == 'ckd\t', 'classification'] = 'ckd'
dataframe.loc[dataframe['classification'] == 'ckd', 'classification'] = 1
dataframe.loc[dataframe['classification'] == 'notckd', 'classification'] = 0
print(dataframe.groupby("classification").size())

classification
0    135
1     98
dtype: int64


In [328]:
X=dataframe.iloc[:,:-1].values
y=dataframe.iloc[:,24].values

#Tratamiento de datos faltantes NaN por medio de sustituirlos por la media
from sklearn.preprocessing import Imputer
imputer = Imputer(missing_values = "NaN", strategy = "mean", axis = 0) #axis = 0 es por columna, axis=1 es por fila
imputer = imputer.fit(X[:,0:5])
X[:,0:5] = imputer.transform(X[:,0:5])
imputer = imputer.fit(X[:,9:18])
X[:,9:18] = imputer.transform(X[:,9:18])

X.shape

//anaconda3/envs/sdc2/lib/python3.7/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


(233, 24)

In [329]:
labelencoder_X = LabelEncoder()
X[:,5] = labelencoder_X.fit_transform(X[:,5])
X[:,6] = labelencoder_X.fit_transform(X[:,6])
X[:,7] = labelencoder_X.fit_transform(X[:,7])
X[:,8] = labelencoder_X.fit_transform(X[:,8])
X[:,18] = labelencoder_X.fit_transform(X[:,18])
X[:,19] = labelencoder_X.fit_transform(X[:,19])
X[:,20] = labelencoder_X.fit_transform(X[:,20])
X[:,21] = labelencoder_X.fit_transform(X[:,21])
X[:,22] = labelencoder_X.fit_transform(X[:,22])
X[:,23] = labelencoder_X.fit_transform(X[:,23])

In [330]:
onehotencoder = OneHotEncoder(categorical_features=[5,6,7,8,18,19,20,21,22,23])
X = onehotencoder.fit_transform(X).toarray()
#for row in X:
#    print(row)

//anaconda3/envs/sdc2/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
//anaconda3/envs/sdc2/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [331]:
model = linear_model.LogisticRegression() 
model.fit(X,y)

//anaconda3/envs/sdc2/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [332]:
predictions = model.predict(X) 
print(predictions)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]


In [333]:
model.score(X,y)

1.0

In [334]:
validation_size = 0.30
seed = 20
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, y, test_size=validation_size, random_state=seed)

In [335]:
prob_predictions = model.predict_proba(X)
print(prob_predictions)

[[1.55203850e-10 1.00000000e+00]
 [4.44774400e-05 9.99955523e-01]
 [1.52174526e-02 9.84782547e-01]
 [9.24791018e-08 9.99999908e-01]
 [7.27579997e-06 9.99992724e-01]
 [1.45509428e-05 9.99985449e-01]
 [9.50448609e-11 1.00000000e+00]
 [3.77029963e-10 1.00000000e+00]
 [1.18771178e-07 9.99999881e-01]
 [1.01243086e-05 9.99989876e-01]
 [2.01622418e-05 9.99979838e-01]
 [7.03792294e-06 9.99992962e-01]
 [8.77472385e-02 9.12252761e-01]
 [1.23207883e-06 9.99998768e-01]
 [1.95829645e-07 9.99999804e-01]
 [4.07766054e-02 9.59223395e-01]
 [2.19693511e-03 9.97803065e-01]
 [1.97541384e-03 9.98024586e-01]
 [1.19381793e-09 9.99999999e-01]
 [2.08373162e-01 7.91626838e-01]
 [2.90497292e-05 9.99970950e-01]
 [1.75488906e-06 9.99998245e-01]
 [6.07593512e-07 9.99999392e-01]
 [7.12832643e-07 9.99999287e-01]
 [6.59646382e-09 9.99999993e-01]
 [1.44601095e-05 9.99985540e-01]
 [1.75172477e-02 9.82482752e-01]
 [5.86369886e-09 9.99999994e-01]
 [3.42085247e-04 9.99657915e-01]
 [6.48527898e-11 1.00000000e+00]
 [9.757502

In [336]:
probs_df = pd.DataFrame(prob_predictions)

In [337]:
X_result = dataframe.reset_index().copy()
X_result["objetivo"] = y.tolist()
X_result["prediccion"] = predictions
X_result = pd.concat([X_result,probs_df],axis=1)
X_result[["objetivo","prediccion",0,1]]

,objetivo,prediccion,0,1
0,1,1,1.552038e-10,1.000000
1,1,1,4.447744e-05,0.999956
2,1,1,1.521745e-02,0.984783
3,1,1,9.247910e-08,1.000000
4,1,1,7.275800e-06,0.999993
5,1,1,1.455094e-05,0.999985
6,1,1,9.504486e-11,1.000000
7,1,1,3.770300e-10,1.000000
8,1,1,1.187712e-07,1.000000
9,1,1,1.012431e-05,0.999990


In [338]:
name='Logistic Regression'
kfold = model_selection.KFold(n_splits=5, random_state=seed) 
cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std()) 
print(msg)

Logistic Regression: 0.975568 (0.012221)


//anaconda3/envs/sdc2/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
//anaconda3/envs/sdc2/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
//anaconda3/envs/sdc2/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
//anaconda3/envs/sdc2/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
//anaconda3/envs/sdc2/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 

In [339]:
predictions = model.predict(X_validation) 
print(accuracy_score(Y_validation, predictions))

1.0


In [340]:
print(confusion_matrix(Y_validation, predictions))

[[43  0]
 [ 0 27]]


In [341]:
print(classification_report(Y_validation, predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        43
           1       1.00      1.00      1.00        27

    accuracy                           1.00        70
   macro avg       1.00      1.00      1.00        70
weighted avg       1.00      1.00      1.00        70

